In [7]:
import librosa
import pandas as pd
import numpy as np
from IPython.lib.display import Audio
from matplotlib import pyplot as plt
import multiprocessing
import scipy.signal
from scipy.fft import fft
from scipy.signal import butter, lfilter
import os
from utils import *
from preprocess import *
from sklearn.model_selection import train_test_split

In [2]:
data_dir = "C:/Users/Jayde/Desktop/library/zebra_finch/data/"
anno = pd.read_csv(f'{data_dir}annotations.csv')

audio = []
srs= []

#adding the audio data to the dataframe
for _, row in anno.iterrows():
    recording, sr = librosa.load(f'{data_dir}vocalizations/{row.fn}')
    audio.append(recording)
    srs.append(sr)
    
anno['audio'] = audio



In [3]:
recs = []
srs = []
file_names= []
call_types = []
names = []


data_dir = "C:/Users/Jayde/Desktop/library/zebra_finch/data/AdultVocalizations/"
for file_name in os.listdir(data_dir):
    recording, sr = librosa.load(f'{data_dir}{file_name}', sr = None)
    #print(file)
    try:
        name, file= file_name.split("_")
        date, call_type, rendition= file.split("-")
        call_type = call_type[:2]
        #print(name, date, call_type)
        recs.append(recording)
        names.append(name)
        file_names.append(file_name)
        call_types.append(call_type)
        srs.append(sr)
    except:
        print(f"skipped {file}")
        continue

print(f"processed {len(recs)} out of {len(os.listdir(data_dir))}")

processed 2969 out of 2969


In [3]:
data_dir = "C:/Users/Jayde/Desktop/library/zebra_finch/data/AdultVocalizations/"
recs = []
#file_names= []
call_types = []
#names = []
slice_len = 16384
n_types = 5
equal = True

for file_name in os.listdir(data_dir):
    recording, sr = lb.load(f'{data_dir}{file_name}', sr = None)
    #print(file)
    try:
        name, file= file_name.split("_")
        date, call_type, rendition= file.split("-")
        call_type = call_type[:2]
        #print(name, date, call_type)
        recording /= np.max(np.abs(recording))
        recs.append(recording)
        #names.append(name)
        #file_names.append(file_name)
        call_types.append(call_type)
    except:
        print(f"skipped {file}")
        continue
df = pd.DataFrame()
df["call_type"]= call_types
df["rec"] = recs
durations = [x.shape[0] / sr for x in recs]
df["duration"] = durations
print(f"obtained {len(os.listdir(data_dir))} samples")
dur = slice_len/sr
df= df[df["duration"] <= dur]

call_type_counts = df["call_type"].value_counts()
top_n = call_type_counts.head(n_types).index.tolist()

print(f"top {n_types} call types for duration <= {dur}: {top_n}")
df_top = df[df["call_type"].isin(top_n)]
if equal:
    min_count = min(call_type_counts.head(n_types).values.tolist())

    def sample_from_group(group):
        return group.sample(min_count, replace=True)
    
    # Apply the sampling function to each group
    df_top = df_top.groupby('call_type', group_keys=False).apply(sample_from_group)

    # Reset the index of the resulting DataFrame
    df_top.reset_index(drop=True, inplace=True)
    call_type_counts = df_top["call_type"].value_counts()
    print(call_type_counts)

audio = [centre_and_pad(signal, slice_len) for signal in df_top["rec"]]
audio = [bandpass_filter(signal,250,12000, sr) for signal in audio]
audio = np.array(audio)
n_train_data = len(audio)
print(f"reduced to {n_train_data} training samlples")
labels = pd.get_dummies(df_top['call_type']).values

obtained 2969 samples
top 5 call types for duration <= 0.37151927437641724: ['Te', 'DC', 'Ne', 'Th', 'Tu']
DC    240
Ne    240
Te    240
Th    240
Tu    240
Name: call_type, dtype: int64
reduced to 1200 training samlples


In [5]:
labels.astype('float32')

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(audio, labels, test_size=0.2, random_state=7)

In [12]:
len(np.argmax(y_train, axis=1))

960

In [4]:
df = pd.DataFrame()
df["file_name"] = file_names
df["name"] = names
df["call_type"]= call_types
#df["sr"] = srs
df["rec"] = recs
durations = [x.shape[0] / 44100 for x in recs]
df["duration"] = durations


In [5]:
df.head()

,file_name,name,call_type,rec,duration
0,BlaBla0506_110302-AggC-04.wav,BlaBla0506,Ag,"[-1.5258789e-05, -0.0009918213, 3.0517578e-05,...",0.308118
1,BlaBla0506_110302-AggC-05.wav,BlaBla0506,Ag,"[0.0001373291, 0.00021362305, 0.00062561035, 0...",0.599184
2,BlaBla0506_110302-DC-01.wav,BlaBla0506,DC,"[0.00015258789, 0.0002746582, -0.0005340576, -...",0.238594
3,BlaBla0506_110302-DC-02.wav,BlaBla0506,DC,"[0.002105713, 0.0018157959, 0.0018157959, 0.00...",0.261111
4,BlaBla0506_110302-DC-05.wav,BlaBla0506,DC,"[-4.5776367e-05, -0.00032043457, -0.0006408691...",0.253220


In [6]:
df.describe()

,duration
count,2969.000000
mean,0.338989
std,0.719706
min,0.032494
25%,0.085442
50%,0.127551
75%,0.246100
max,11.705079


In [7]:
slice_lens = [4096, 16384, 32768, 65536]
for slice_len in slice_lens:
    dur = slice_len/44100
    count = (df["duration"]<= dur).sum()
    print(f" max dur:{dur} for slice len {slice_len}. {count} recordings ")

 max dur:0.09287981859410431 for slice len 4096. 900 recordings 
 max dur:0.37151927437641724 for slice len 16384. 2542 recordings 
 max dur:0.7430385487528345 for slice len 32768. 2699 recordings 
 max dur:1.486077097505669 for slice len 65536. 2807 recordings 


In [8]:
df= df[df["duration"] <= 16384/44100]

In [8]:
call_type_counts = df["call_type"].value_counts()
top_8 = call_type_counts.head(8).index.tolist()
top_8

['Te', 'DC', 'Ne', 'Th', 'Tu', 'Ag', 'Wh', 'LT']

In [9]:
df_top = df[df["call_type"].isin(top_8)]
df_top["call_type"].value_counts()

Te    613
DC    597
Ne    468
Th    301
Tu    240
Ag    164
Wh    131
LT     16
Name: call_type, dtype: int64

In [10]:
slice_lens = [16384, 32768, 65536]
for slice_len in slice_lens:
    dur = slice_len/44100
    count = (df_top["duration"]<= dur).sum()
    train_size = (count//64) *64
    print(f" max dur:{dur} for slice len {slice_len}. {count} recordings -> train size {train_size}")

 max dur:0.37151927437641724 for slice len 16384. 2530 recordings -> train size 2496
 max dur:0.7430385487528345 for slice len 32768. 2530 recordings -> train size 2496
 max dur:1.486077097505669 for slice len 65536. 2530 recordings -> train size 2496


In [11]:
df_top["call_type"].value_counts()

Te    613
DC    597
Ne    468
Th    301
Tu    240
Ag    164
Wh    131
LT     16
Name: call_type, dtype: int64

In [12]:
def bandpass_filter(data, low, high, sr, order=5):
    nyquist = sr/2
    low = low/nyquist
    high = high/nyquist
    b, a = butter(order, [low, high], btype='band')
    filtered = lfilter(b, a, data)
    return filtered


In [ ]:
signal = df_top["rec"][2]
print(signal)
sr = 44100
create_plots(signal,sr)
filtered = bandpass_filter(signal,250,12000,sr)
create_plots(filtered,sr)
centred = centre_and_pad(filtered)
normalised = z_score_normalise(centred)
print(normalised)
create_plots(normalised, sr, amp_envelope=False)
t, zcr = ZCR(filtered,frame_size = 128, hop_length= 64, sr= sr)
plt.plot(t,zcr)
plt.show()

In [ ]:
IPython.display.display(Audio(signal, rate =sr))
IPython.display.display(Audio(normalised, rate =sr))

In [34]:
def centre_and_pad(signal, slice_len= 16384):
    nsamples = len(signal)
    l_pad = (slice_len - nsamples)//2
    r_pad = slice_len - (nsamples + l_pad)
    if len(signal) < slice_len:
        padded_array = np.pad(signal,
                              (l_pad,r_pad),
                              mode = "constant")
        return padded_array
    else:
        signal = signal[:slice_len]
    return signal

In [ ]:
#envelope features
t, ae = amplitude_envelope(filtered, 128,64, sr)

#normalise so sums of envelope values = 1?

#temporal is amplittude envelope
t_mean = np.mean(ae)

#standard deviation
temporal_duration = np.std(ae)

#skewness
from scipy.stats import skew
t_skew = skew(ae, bias = True)

#kurtosis
from scipy.stats import kurtosis 
t_kurt = kurtosis(ae, bias = True)

#entropy 
from scipy.stats import entropy
t_entropy = entropy(ae)

In [28]:
def RMS_energy(signal, frame_size, hop_length, sr):
    rms_energy = librosa.feature.rms(y=signal,frame_length=frame_size, hop_length=hop_length)[0]
    frames = range(0,rms_energy.size)
    t = lb.frames_to_time(frames, hop_length= hop_length, sr = sr)
    return t,rms_energy

In [14]:
def ZCR(signal, frame_size, hop_length, sr):
    zcr = librosa.feature.zero_crossing_rate(y=signal,frame_length=frame_size, hop_length=hop_length)[0]
    frames = range(0,zcr.size)
    t = lb.frames_to_time(frames, hop_length= hop_length, sr = sr)
    return t,zcr

In [29]:
def create_plots(signal, sr, window_length = 1024, save_dir = None, lw=0.6, fmax=6e3, font_size = 13, amp_envelope= True, plot_rms = True):
    fmax = sr/2 #nyquist frequency 
    N = len(signal)
    delta_t = 1 / sr
    times = np.arange(0, N) / sr
    signalf = fft(signal)
    freqs = np.linspace(0.0, 1.0/(2.0*delta_t), N//2)
    
    fig, axs = plt.subplots(1,3, figsize=(20,5))
    axs[0].plot(times, signal, linewidth=lw)
    #axs[0].librosa.display.waveshow(code_1[1], sr=sr)
    axs[0].set_xlabel('Time (s)', fontsize= font_size)
    axs[0].set_ylabel('Amplitude', fontsize = font_size)
    t_ae, ae = amplitude_envelope(signal,frame_size = 128, hop_length= 64, sr= sr)
    t_rms, rms = RMS_energy(signal,frame_size = 128, hop_length= 64, sr= sr)

    axs[0].plot(t_ae,ae, color = "r") if amp_envelope else None
    axs[0].plot(t_rms,rms, color = "r") if plot_rms else None
    axs[0].set_xlim(0,max(times))
    #axs[0].set_title('Time Domain Representation')
    #axs[0].xticks(fontsize = font_size)
    #axs[0].yticks(fontsize= font_size)
    
    axs[1].plot(freqs, 2.0/N * np.abs(signalf[0:N//2]), linewidth=0.4)
    axs[1].set_xlabel('Frequency (Hz)', fontsize= font_size)
    axs[1].set_ylabel('Amplitude', fontsize= font_size)
    #axs[1].set_title('Frequency Domain Representation')
    axs[1].set_xlim([0, fmax])
    #axs[1].xticks(fontsize = font_size)
    #axs[1].yticks(fontsize= font_size)


    f_bins, t_bins, Sxx = spectrogram(signal, fs=sr,
                                 window='hann', nperseg=window_length,
                                 noverlap=window_length-64, detrend=False,
                                 scaling='spectrum')
    im = axs[2].pcolormesh(t_bins, f_bins, 20*np.log10(Sxx+1e-100), cmap='magma')

    axs[2].set_xlabel('Time (s)', fontsize= font_size)
    axs[2].set_ylabel('Frequency (Hz)', fontsize= font_size)
    #axs[2].set_xlim(0,1)
    #axs[2].xticks(fontsize = font_size)
    #axs[2].yticks(fontsize= font_size)
    #axs[2].set_title(f'STFT with nperseg={window_length}')

    # Add colorbar
    cbar = fig.colorbar(im, ax=axs[2])
    cbar.set_label('Amplitude (dB)',fontsize= font_size)
    for ax in axs:
        ax.tick_params(axis='both', labelsize=font_size-2)
    
    if save_dir:
        plt.savefig(save_dir, dpi= 300, bbox_inches = "tight", transparent = True)
    plt.show()